In [34]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [35]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

$$
    r_{u, i} = f(user\_features, item\_features)
$$

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [46]:
# DO NOT USE, WORKS POORLY

# ONE HOT ENCODING

def prepare_users_df(interactions_df):
    interactions_df = interactions_df.copy().drop_duplicates(subset='user_id')
    one_hot = pd.get_dummies(data=interactions_df, columns=['term', 'room_segment', 'rate_plan', 'length_of_stay_bucket', 'n_people_bucket', 'weekend_stay'], prefix='user')
    users_df = one_hot.drop(['item_id'], axis = 1, errors='ignore')
    user_features = list(users_df.columns.values[1:])
    
    return users_df, user_features

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

user_avg = users_df.mean()

['user_WinterVacation', 'user_Easter', 'user_OffSeason', 'user_HighSeason', 'user_LowSeason', 'user_MayLongWeekend', 'user_NewYear', 'user_Christmas', 'user_[0-160]', 'user_[160-260]', 'user_[260-360]', 'user_[360-500]', 'user_[500-900]', 'user_Standard', 'user_Nonref', 'user_[0-1]', 'user_[2-3]', 'user_[4-7]', 'user_[8-inf]', 'user_[1-1]', 'user_[2-2]', 'user_[3-4]', 'user_[5-inf]', 'user_True', 'user_False']


,user_id,user_WinterVacation,user_Easter,user_OffSeason,user_HighSeason,user_LowSeason,user_MayLongWeekend,user_NewYear,user_Christmas,user_[0-160],user_[160-260],user_[260-360],user_[360-500],user_[500-900],user_Standard,user_Nonref,user_[0-1],user_[2-3],user_[4-7],user_[8-inf],user_[1-1],user_[2-2],user_[3-4],user_[5-inf],user_True,user_False
0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0
42,50,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0
333,706,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1
569,115,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1
1350,1736,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
2967,96,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0
12667,7779,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0


In [45]:
# DO NOT USE, WORKS POORLY

# ONE HOT ENCODING + AVERAGE VALUE FOR BUCKETS

def prepare_users_df(interactions_df):
    
    interactions_copy = interactions_df.copy()
    
    # Calculate average from buckets
    room_segment = {'[0-160]' : 80, '[160-260]' : 220, '[260-360]' : 320, '[360-500]' : 430, '[500-900]' : 700}
    length_of_stay_bucket = {'[0-1]' : 0.5, '[2-3]' : 2.5, '[4-7]' : 6, '[8-inf]' : 11}
    n_people_bucket = {'[1-1]' : 1, '[2-2]' : 2, '[3-4]' : 3.5, '[5-inf]' : 8}
    
    
    interactions_copy.loc[:, 'room_segment'] = interactions_copy['room_segment'].replace(room_segment)  
    interactions_copy.loc[:, "length_of_stay_bucket"] = interactions_copy["length_of_stay_bucket"].replace(length_of_stay_bucket)  
    interactions_copy.loc[:, "n_people_bucket"] = interactions_copy["n_people_bucket"].replace(n_people_bucket)
    
    users_grouped = interactions_copy.loc[:, ["user_id"] + ["room_segment"] + ["length_of_stay_bucket"] + ["n_people_bucket"]].groupby("user_id").mean().fillna(0).astype(int)
    
    interactions_copy = interactions_copy.copy().drop_duplicates(subset='user_id')
    
    interactions_copy = interactions_copy.drop(columns=['room_segment', 'n_people_bucket', 'length_of_stay_bucket'])
    
    one_hot = pd.get_dummies(data=interactions_copy, columns=['term', 'rate_plan', 'weekend_stay'], prefix='user')
    
    one_hot['count'] = 1
    
    one_hot = one_hot.groupby("user_id").sum().reset_index()
    
    one_hot = users_grouped.merge(one_hot, on="user_id")
    
    users_df = one_hot.drop(['item_id'], axis = 1, errors='ignore')
    users_df = users_df.drop(['count'], axis = 1, errors='ignore')
    user_features = list(users_df.columns.values[1:])
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

user_avg = users_df.mean()


['room_segment', 'length_of_stay_bucket', 'n_people_bucket', 'user_WinterVacation', 'user_Easter', 'user_OffSeason', 'user_HighSeason', 'user_LowSeason', 'user_MayLongWeekend', 'user_NewYear', 'user_Christmas', 'user_Standard', 'user_Nonref', 'user_True', 'user_False']


,user_id,room_segment,length_of_stay_bucket,n_people_bucket,user_WinterVacation,user_Easter,user_OffSeason,user_HighSeason,user_LowSeason,user_MayLongWeekend,user_NewYear,user_Christmas,user_Standard,user_Nonref,user_True,user_False
0,1,233,3,2,1,0,0,0,0,0,0,0,1,0,1,0
40,50,263,2,4,0,0,1,0,0,0,0,0,1,0,1,0
84,96,222,2,3,0,0,0,1,0,0,0,0,0,1,1,0
102,115,270,4,4,0,0,1,0,0,0,0,0,1,0,0,1
371,706,225,3,3,1,0,0,0,0,0,0,0,1,0,0,1
1383,1736,226,2,1,0,0,1,0,0,0,0,0,0,1,1,0
7301,7779,220,6,4,0,0,1,0,0,0,0,0,1,0,1,0


In [48]:
# CALCULATE PROBABILITIES AMONG USER'S VALUES

# WORKS GREAT :)

def prepare_users_df(interactions_df):
    
    interactions_copy = interactions_df.copy()
    
    one_hot = pd.get_dummies(data=interactions_copy, columns=['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay'], prefix='user')
    
    one_hot['count'] = 1
    
    one_hot = one_hot.groupby("user_id").sum().reset_index()
    
    one_hot.loc[:, 'user_WinterVacation':'user_False'] = one_hot.loc[:, 'user_WinterVacation':'user_False'].div(one_hot["count"],axis=0)
    
    users_df = one_hot.drop(['item_id'], axis = 1, errors='ignore')
    users_df = users_df.drop(['count'], axis = 1, errors='ignore')
    user_features = list(users_df.columns.values[1:])
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(HTML(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15).to_html()))

# CALCULATE AVERAGE USER TO FILL NAN VALUES LATER

user_avg = users_df.mean()


['user_WinterVacation', 'user_Easter', 'user_OffSeason', 'user_HighSeason', 'user_LowSeason', 'user_MayLongWeekend', 'user_NewYear', 'user_Christmas', 'user_[0-1]', 'user_[2-3]', 'user_[4-7]', 'user_[8-inf]', 'user_Standard', 'user_Nonref', 'user_[0-160]', 'user_[160-260]', 'user_[260-360]', 'user_[360-500]', 'user_[500-900]', 'user_[1-1]', 'user_[2-2]', 'user_[3-4]', 'user_[5-inf]', 'user_True', 'user_False']


,user_id,user_WinterVacation,user_Easter,user_OffSeason,user_HighSeason,user_LowSeason,user_MayLongWeekend,user_NewYear,user_Christmas,user_[0-1],user_[2-3],user_[4-7],user_[8-inf],user_Standard,user_Nonref,user_[0-160],user_[160-260],user_[260-360],user_[360-500],user_[500-900],user_[1-1],user_[2-2],user_[3-4],user_[5-inf],user_True,user_False
0,1,0.090909,0.0,0.681818,0.090909,0.136364,0.000000,0.000000,0.0,0.000000,0.590909,0.409091,0.000000,0.500000,0.500000,0.000000,0.863636,0.136364,0.0,0.0,0.000000,0.727273,0.181818,0.090909,0.772727,0.227273
40,50,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.0,0.000000,0.913043,0.086957,0.000000,0.260870,0.739130,0.000000,0.565217,0.434783,0.0,0.0,0.000000,0.173913,0.521739,0.304348,0.782609,0.217391
84,96,0.090909,0.0,0.681818,0.136364,0.045455,0.045455,0.000000,0.0,0.272727,0.636364,0.045455,0.045455,0.272727,0.727273,0.045455,0.863636,0.090909,0.0,0.0,0.045455,0.272727,0.590909,0.090909,0.727273,0.272727
102,115,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000,0.500000,0.000000,1.000000,0.000000,0.000000,0.500000,0.500000,0.0,0.0,0.500000,0.000000,0.000000,0.500000,0.000000,1.000000
371,706,0.095238,0.0,0.511905,0.190476,0.142857,0.047619,0.011905,0.0,0.297619,0.380952,0.238095,0.083333,0.976190,0.023810,0.035714,0.857143,0.107143,0.0,0.0,0.119048,0.154762,0.583333,0.130952,0.583333,0.416667
1383,1736,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.0,0.241379,0.551724,0.206897,0.000000,0.172414,0.827586,0.000000,0.931034,0.068966,0.0,0.0,0.379310,0.413793,0.206897,0.000000,0.448276,0.551724
7301,7779,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.250000,0.500000,0.250000,1.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.750000,0.250000,0.750000,0.250000


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [50]:
# ONE HOT ENCODING

# WORKS GREAT :)

def prepare_items_df(interactions_df):
    interactions_df = interactions_df.copy().drop_duplicates(subset='item_id')
    one_hot = pd.get_dummies(data=interactions_df, columns=['term', 'room_segment', 'rate_plan', 'length_of_stay_bucket', 'n_people_bucket', 'weekend_stay'], prefix='item')
    items_df = one_hot.drop(['user_id'], axis = 1, errors='ignore')
    item_features = list(items_df.columns.values[1:])
    
    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)
print(item_features)

display(HTML(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15).to_html()))

['item_WinterVacation', 'item_Easter', 'item_OffSeason', 'item_HighSeason', 'item_LowSeason', 'item_MayLongWeekend', 'item_NewYear', 'item_Christmas', 'item_[0-160]', 'item_[160-260]', 'item_[260-360]', 'item_[360-500]', 'item_[500-900]', 'item_Standard', 'item_Nonref', 'item_[0-1]', 'item_[2-3]', 'item_[4-7]', 'item_[8-inf]', 'item_[1-1]', 'item_[2-2]', 'item_[3-4]', 'item_[5-inf]', 'item_True', 'item_False']


,item_id,item_WinterVacation,item_Easter,item_OffSeason,item_HighSeason,item_LowSeason,item_MayLongWeekend,item_NewYear,item_Christmas,item_[0-160],item_[160-260],item_[260-360],item_[360-500],item_[500-900],item_Standard,item_Nonref,item_[0-1],item_[2-3],item_[4-7],item_[8-inf],item_[1-1],item_[2-2],item_[3-4],item_[5-inf],item_True,item_False
0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [51]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos

        self.recommender_df = pd.DataFrame(
            columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None

        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """

        interactions_df = interactions_df.copy()

        # Prepare users_df and items_df

        users_df, user_features = prepare_users_df(interactions_df)

        self.users_df = users_df
        self.user_features = user_features

        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]

        # Generate negative interactions

        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]

        interactions_df.loc[:, 'interacted'] = 1

        negative_interactions = []

        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.

        pos_tuples = interactions_df.loc[:,  ['user_id', 'item_id']]
        
        # Iterate over all items using tuples
        pos_tuples = set(pos_tuples.itertuples(index=False, name=None))
        pos_tuples = {key: 1 for key in pos_tuples}

        user_ids, item_ids = interactions_df.user_id.unique(), interactions_df.item_id.unique()

        while len(negative_interactions) < self.n_neg_per_pos * len(interactions_df):
            x, y = np.random.choice(user_ids), np.random.choice(item_ids)

            if (x, y) in pos_tuples:
                continue
            negative_interactions.append((x, y, 0))

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])

        # Get the input data for the model

        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])

        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.

        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """

        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]

        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.

        users_df = pd.merge(users_df, self.users_df,
                            on=['user_id'], how='left')
        users_df = users_df.fillna(user_avg)
        items_df, item_features = prepare_items_df(items_df)

        # Score the items

        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])

        for ix, user in users_df.iterrows():

            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            user_df = user.to_frame().transpose()
            carthesian = pd.merge(items_df, user_df, how='cross')

            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            scores = self.model.predict(
                carthesian.loc[:, user_features + item_features].values)

            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.

            chosen_ids = (-scores).argsort()[:n_recommendations]

            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )

            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat(
                [self.recommender_df, user_recommendations])

        return self.recommender_df


class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()


class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)


class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)


class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)


# Quick test of the recommender

In [52]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [57]:
# Fit method
cb_user_item_recommender = ContentBasedUserItemRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [58]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(10))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,51,0.589844,OffSeason,[2-3],Nonref,[160-260],[3-4],True
1,1.0,22,0.583984,OffSeason,[2-3],Standard,[160-260],[3-4],True
2,1.0,55,0.549805,OffSeason,[2-3],Nonref,[160-260],[2-2],True
3,1.0,32,0.543945,OffSeason,[2-3],Standard,[160-260],[2-2],True
4,1.0,57,0.510742,OffSeason,[2-3],Nonref,[160-260],[3-4],False
5,1.0,34,0.504883,OffSeason,[2-3],Standard,[160-260],[3-4],False
6,1.0,62,0.485352,OffSeason,[2-3],Nonref,[160-260],[5-inf],True
7,1.0,78,0.480469,HighSeason,[2-3],Nonref,[160-260],[3-4],True
8,1.0,76,0.480469,OffSeason,[4-7],Nonref,[160-260],[3-4],True
9,1.0,31,0.479492,OffSeason,[2-3],Standard,[160-260],[5-inf],True


In [59]:
# Fit method
cb_user_item_recommender = LinearRegressionCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [60]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(10))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,51,0.589844,OffSeason,[2-3],Nonref,[160-260],[3-4],True
1,1.0,22,0.582520,OffSeason,[2-3],Standard,[160-260],[3-4],True
2,1.0,55,0.551758,OffSeason,[2-3],Nonref,[160-260],[2-2],True
3,1.0,32,0.544922,OffSeason,[2-3],Standard,[160-260],[2-2],True
4,1.0,57,0.512695,OffSeason,[2-3],Nonref,[160-260],[3-4],False
5,1.0,34,0.505859,OffSeason,[2-3],Standard,[160-260],[3-4],False
6,1.0,62,0.486328,OffSeason,[2-3],Nonref,[160-260],[5-inf],True
7,1.0,76,0.482422,OffSeason,[4-7],Nonref,[160-260],[3-4],True
8,1.0,31,0.479004,OffSeason,[2-3],Standard,[160-260],[5-inf],True
9,1.0,78,0.476562,HighSeason,[2-3],Nonref,[160-260],[3-4],True


In [61]:
# Fit method
cb_user_item_recommender = XGBoostCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [62]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(10))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,173,2.895737e+145,OffSeason,[0-1],Standard,[160-260],[3-4],True
1,1.0,423,2.895737e+145,OffSeason,[0-1],Nonref,[160-260],[2-2],True
2,1.0,273,2.895737e+145,OffSeason,[0-1],Nonref,[160-260],[3-4],True
3,1.0,54,2.895737e+145,OffSeason,[0-1],Standard,[160-260],[2-2],True
4,1.0,381,2.895000e+145,LowSeason,[2-3],Standard,[0-160],[1-1],False
5,1.0,526,2.895000e+145,WinterVacation,[2-3],Nonref,[0-160],[1-1],True
6,1.0,390,2.895000e+145,LowSeason,[0-1],Nonref,[260-360],[1-1],True
7,1.0,402,2.895000e+145,LowSeason,[2-3],Nonref,[360-500],[1-1],True
8,1.0,403,2.895000e+145,LowSeason,[2-3],Nonref,[260-360],[1-1],True
9,1.0,404,2.895000e+145,LowSeason,[2-3],Standard,[260-360],[1-1],False


In [63]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [64]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(10))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,53,0.999130,OffSeason,[2-3],Nonref,[160-260],[2-2],False
1,1.0,55,0.998904,OffSeason,[2-3],Nonref,[160-260],[2-2],True
2,1.0,51,0.998516,OffSeason,[2-3],Nonref,[160-260],[3-4],True
3,1.0,61,0.992451,OffSeason,[4-7],Nonref,[160-260],[2-2],True
4,1.0,22,0.992076,OffSeason,[2-3],Standard,[160-260],[3-4],True
5,1.0,23,0.986856,OffSeason,[4-7],Standard,[160-260],[2-2],True
6,1.0,32,0.979718,OffSeason,[2-3],Standard,[160-260],[2-2],True
7,1.0,21,0.968531,OffSeason,[2-3],Standard,[160-260],[2-2],False
8,1.0,33,0.966890,OffSeason,[2-3],Standard,[260-360],[2-2],True
9,1.0,56,0.965730,OffSeason,[4-7],Nonref,[160-260],[2-2],False


In [165]:

# DO NOT RUN 65 MIN

cb_user_item_recommender = SVRCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [166]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations.head(10))

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,55,1.132122,OffSeason,[2-3],Nonref,[160-260],[2-2],True
1,1.0,32,1.114019,OffSeason,[2-3],Standard,[160-260],[2-2],True
2,1.0,23,0.986181,OffSeason,[4-7],Standard,[160-260],[2-2],True
3,1.0,61,0.971776,OffSeason,[4-7],Nonref,[160-260],[2-2],True
4,1.0,51,0.944905,OffSeason,[2-3],Nonref,[160-260],[3-4],True
5,1.0,22,0.930947,OffSeason,[2-3],Standard,[160-260],[3-4],True
6,1.0,53,0.853980,OffSeason,[2-3],Nonref,[160-260],[2-2],False
7,1.0,14,0.837589,OffSeason,[4-7],Standard,[160-260],[3-4],True
8,1.0,21,0.826649,OffSeason,[2-3],Standard,[160-260],[2-2],False
9,1.0,76,0.806991,OffSeason,[4-7],Nonref,[160-260],[3-4],True


# Tuning method

In [66]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [67]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

### Test recommenders with default params

#### LINEAR

In [69]:
cb_user_item_recommender = LinearRegressionCBUIRecommender()

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [70]:
display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130502


#### RANDOM FOREST

In [81]:
cb_user_item_recommender = RandomForestCBUIRecommender()
# Give the name of your recommender in the line below
RandomForrest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

RandomForrest_cbui_tts_results = pd.DataFrame(
    RandomForrest_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [91]:
display(HTML(RandomForrest_cbui_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.056687,0.111337,0.131365,0.21962,0.056687,0.089212,0.097346,0.124859


#### XGBOOST

In [73]:
cb_user_item_recommender = XGBoostCBUIRecommender()
# Give the name of your recommender in the line below
XGBoost_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

XGBoost_cbui_tts_results = pd.DataFrame(
    XGBoost_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])


In [74]:
display(HTML(XGBoost_cbui_tts_results.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.001358,0.006449,0.009504,0.017312,0.001358,0.004126,0.005367,0.007898


#### SVR

In [92]:
# DO NOT RUN 275 min

cb_user_item_recommender2 = SVRCBUIRecommender(
    **{'C': 1.0, 'epsilon': 0.1, 'kernel': 'rbf'}
)
# Give the name of your recommender in the line below
SVR_cbui_tts_results2 = [['SVRCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender2, interactions_df, items_df))]

SVR_cbui_tts_results2 = pd.DataFrame(
    SVR_cbui_tts_results2, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(SVR_cbui_tts_results2.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.059403,0.144603,0.193822,0.265784,0.059403,0.107203,0.12788,0.150752


In [94]:
display(SVR_cbui_tts_results2)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.059403,0.144603,0.193822,0.265784,0.059403,0.107203,0.12788,0.150752


#### AMAZON

In [76]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [77]:
display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468


In [160]:
recommenders_list = []
recommenders_list.append(linear_cbui_tts_results)
recommenders_list.append(RandomForrest_cbui_tts_results)
recommenders_list.append(XGBoost_cbui_tts_results)
recommenders_list.append(amazon_tts_results)
recommenders_list.append(SVR_cbui_tts_results2)

tts_results = pd.concat(recommenders_list).reset_index(drop=True)
display(tts_results.sort_values(by=['HR@10']))


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
2,XGBoostCBUIRecommender,0.001358,0.006449,0.009504,0.017312,0.001358,0.004126,0.005367,0.007898
1,RandomForestCBUIRecommender,0.056687,0.111337,0.131365,0.219620,0.056687,0.089212,0.097346,0.124859
3,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130502
4,SVRCBUIRecommender,0.059403,0.144603,0.193822,0.265784,0.059403,0.107203,0.127880,0.150752


## TUNING

# LinearRegression

In [96]:
# # OK

# param_space = {
#     'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
# }

# best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
#                                   param_space, max_evals=10, show_progressbar=True, seed=seed)

# print("Best parameters:")
# print(best_param_set)

100%|██████████| 10/10 [22:41<00:00, 136.16s/trial, best loss: -0.2264071096064325]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130548


Best parameters:
{'n_neg_per_pos': 5.0}


In [102]:
best_param_set = {'n_neg_per_pos': 5.0}

In [97]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(**best_param_set)

# Give the name of your recommender in the line below
linear_cbui_tts_results_tuned = [['LinearRegressionCBUIRecommenderTuned'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results_tuned = pd.DataFrame(
    linear_cbui_tts_results_tuned, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [155]:
display(HTML(linear_cbui_tts_results_tuned.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommenderTuned,0.049219,0.130007,0.174134,0.243381,0.049219,0.094502,0.113016,0.135005


# RandomForest

In [137]:
# param_space = {
#     'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
#     'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
#     'max_depth': hp.quniform('max_depth', 2, 10, 1),
#     'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
# }

# best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
#                                   param_space, max_evals=100, show_progressbar=True, seed=seed)

# print("Best parameters:")
# print(best_param_set)

100%|██████████| 100/100 [5:38:43<00:00, 203.24s/trial, best loss: -0.23487092678798138] 


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.051595,0.137475,0.182281,0.244739,0.051595,0.100579,0.119385,0.139151


Best parameters:
{'max_depth': 10.0, 'min_samples_split': 8.0, 'n_estimators': 112.0, 'n_neg_per_pos': 10.0}


In [141]:
best_param_set = {'max_depth': 10.0, 'min_samples_split': 8.0, 'n_estimators': 112.0, 'n_neg_per_pos': 10.0}

In [142]:
cb_user_item_recommender = RandomForestCBUIRecommender(**best_param_set)
# Give the name of your recommender in the line below
RandomForrest_cbui_tts_results_tuned = [['RandomForestCBUIRecommenderTuned'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

RandomForrest_cbui_tts_results_tuned = pd.DataFrame(
    RandomForrest_cbui_tts_results_tuned, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [156]:
display(HTML(RandomForrest_cbui_tts_results_tuned.to_html()))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommenderTuned,0.037678,0.125594,0.181602,0.24372,0.037678,0.086836,0.110452,0.130318


# XGBoost

In [103]:
# # This tuning may take around 12 hours

# param_space = {
#     'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
#     'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
#     'max_depth': hp.quniform('max_depth', 2, 10, 1),
#     'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
#     'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
# }

# best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
#                                   param_space, max_evals=300, show_progressbar=True, seed=seed)

# print("Best parameters:")
# print(best_param_set)

100%|██████████| 300/300 [13:43:56<00:00, 164.79s/trial, best loss: -0.23656369022429116]  


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.047862,0.110319,0.158181,0.247115,0.047862,0.085446,0.10533,0.133754


Best parameters:
{'learning_rate': 0.049287321236651666, 'max_depth': 6.0, 'min_samples_split': 12.0, 'n_estimators': 208.0, 'n_neg_per_pos': 9.0}


In [130]:
best_param_set = {'learning_rate': 0.049287321236651666, 'max_depth': 6.0, 'min_samples_split': 12.0, 'n_estimators': 208.0, 'n_neg_per_pos': 9.0}

In [125]:
cb_user_item_recommender = XGBoostCBUIRecommender(**best_param_set)
# Give the name of your recommender in the line below
XGBoost_cbui_tts_results_tuned = [['XGBoostCBUIRecommenderTuned'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

XGBoost_cbui_tts_results_tuned = pd.DataFrame(
    XGBoost_cbui_tts_results_tuned, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

In [157]:
display(XGBoost_cbui_tts_results_tuned)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommenderTuned,0.04888,0.132722,0.154786,0.254243,0.04888,0.097156,0.106123,0.138937


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

# ALL RECOMMENDERS

In [163]:
recommenders_list2 = []
recommenders_list2.append(XGBoost_cbui_tts_results_tuned)
recommenders_list2.append(RandomForrest_cbui_tts_results_tuned)
recommenders_list2.append(linear_cbui_tts_results_tuned)
recommenders_list2.append(amazon_tts_results)
recommenders_list2.append(XGBoost_cbui_tts_results)
recommenders_list2.append(RandomForrest_cbui_tts_results)
recommenders_list2.append(linear_cbui_tts_results)
recommenders_list2.append(SVR_cbui_tts_results2)

tts_results2 = pd.concat(recommenders_list2).reset_index(drop=True)
display(tts_results2.sort_values(by=['HR@10']))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
4,XGBoostCBUIRecommender,0.001358,0.006449,0.009504,0.017312,0.001358,0.004126,0.005367,0.007898
5,RandomForestCBUIRecommender,0.056687,0.111337,0.131365,0.219620,0.056687,0.089212,0.097346,0.124859
3,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468
6,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130502
2,LinearRegressionCBUIRecommenderTuned,0.049219,0.130007,0.174134,0.243381,0.049219,0.094502,0.113016,0.135005
1,RandomForestCBUIRecommenderTuned,0.037678,0.125594,0.181602,0.243720,0.037678,0.086836,0.110452,0.130318
0,XGBoostCBUIRecommenderTuned,0.048880,0.132722,0.154786,0.254243,0.048880,0.097156,0.106123,0.138937
7,SVRCBUIRecommender,0.059403,0.144603,0.193822,0.265784,0.059403,0.107203,0.127880,0.150752


# TUNED RECOMMENDERS

In [164]:
recommenders_list3 = []
recommenders_list3.append(XGBoost_cbui_tts_results_tuned)
recommenders_list3.append(RandomForrest_cbui_tts_results_tuned)
recommenders_list3.append(linear_cbui_tts_results_tuned)
recommenders_list3.append(amazon_tts_results)


tts_results3 = pd.concat(recommenders_list3).reset_index(drop=True)
display(tts_results3.sort_values(by=['HR@10']))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
3,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468
2,LinearRegressionCBUIRecommenderTuned,0.049219,0.130007,0.174134,0.243381,0.049219,0.094502,0.113016,0.135005
1,RandomForestCBUIRecommenderTuned,0.037678,0.125594,0.181602,0.243720,0.037678,0.086836,0.110452,0.130318
0,XGBoostCBUIRecommenderTuned,0.048880,0.132722,0.154786,0.254243,0.048880,0.097156,0.106123,0.138937
